In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = './body_vectors_drags.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_56,dim_57,dim_58,dim_59,dim_60,dim_61,dim_62,dim_63,dim_64,drag
61,-6.639910,-2.575970,-4.333867,0.348634,-6.591602,4.137893,3.448946,9.801783,-4.944950,-1.340411,...,4.211274,0.876610,2.535916,4.460216,7.625662,5.399125,2.165930,1.335564,-6.787674,0.375
354,5.044733,2.543681,-3.247805,-5.378260,0.549456,-1.075264,6.259791,5.443143,1.828774,5.774058,...,5.602760,-0.438914,-1.589318,4.962416,2.475589,-6.530942,-0.353212,1.797986,-2.464886,0.374
358,-0.939149,0.704528,0.749587,-1.643254,-3.556550,0.293420,3.039790,-3.084175,-3.810488,3.759489,...,-4.860520,-0.330673,2.862438,-0.245570,2.726983,-3.573323,-3.095024,0.883057,-0.973858,0.435
275,-1.177986,1.175732,-1.834498,1.306601,-0.493694,1.298445,2.197183,4.541283,1.859666,3.284920,...,-2.512394,-5.162504,0.902922,7.131485,-3.511894,4.393147,-5.885215,0.389388,5.135883,0.437
18,0.390329,-1.530953,-2.521950,-4.864147,-0.742480,0.146031,-0.156536,1.740131,-2.943921,5.280754,...,1.704322,5.339616,3.019771,6.336881,1.606373,-2.136702,-7.156015,3.795955,1.753440,0.367


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [3]:
import os
save_path = './agModels-body'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
Saving ./agModels-body/learner.pkl
Saving ./agModels-body/predictor.pkl
Beginning A

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_55,dim_56,dim_57,dim_58,dim_59,dim_60,dim_61,dim_62,dim_63,dim_64
46,3.262418,4.623484,5.207058,1.474604,5.149589,1.366774,7.350364,4.988407,-4.451728,2.715955,...,3.239450,0.271440,-1.450751,6.201051,2.322523,-0.651795,-2.503192,-1.329733,5.614099,4.784914
101,-1.815653,0.467063,-0.713573,-2.519432,-6.833046,3.925573,3.131740,2.785505,-0.169789,3.797550,...,8.718956,-4.833921,-1.387358,1.010315,0.467406,-7.262593,0.406025,-4.430325,-4.493312,3.210908
175,-0.128397,-0.347771,-4.491439,-0.143762,-1.139696,1.523660,4.283424,1.275272,-3.277825,-0.944104,...,2.371663,-0.860736,-2.739771,4.489244,4.020984,1.973119,3.365454,-2.144289,1.310637,-1.411514
9,-0.334880,-0.358754,-5.121314,-1.869275,0.053525,2.998930,-0.482418,1.283854,-1.355309,-0.988547,...,4.671710,-1.422821,0.833765,5.060172,3.785773,1.105779,2.158277,-2.020028,-0.746978,1.548296
136,0.339348,2.649365,-1.346519,0.872298,-2.672357,1.983254,2.075392,6.573480,-0.809723,-2.231186,...,0.290699,-0.446408,-5.342670,5.205126,5.651273,-2.507123,1.437673,-3.640510,-1.535356,4.507751


In [6]:
%%capture log_output
%config InlineBackend.figure_format = 'retina'
%config Application.log_level = 'DEBUG'
%config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

with open('./output_body.log', 'w') as f:
    f.write(log_output.stdout)

Loading: ./agModels-body/predictor.pkl
Loading: ./agModels-body/learner.pkl
Loading: ./agModels-body/models/trainer.pkl
Loading: ./agModels-body/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-body/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-body/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-body/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-body/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-body/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-body/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-body/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-body/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-body/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-body/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-body/models/CatBoost_BAG_L2/model.pkl
Loading: ./agModels-body/models/ExtraTreesMSE_BAG_L2/model.pkl
Loading: ./agModels-body/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agMo

In [10]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 64 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [11]:
train_data_pred = predictor.predict(train_data, model='WeightedEnsemble_L2')
for item in train_data_pred:
    print(item)

Loading: ./agModels-body/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-body/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-body/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-body/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-body/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-body/models/WeightedEnsemble_L2/model.pkl


0.3881307542324066
0.38251906633377075
0.4216409921646118
0.42200013995170593
0.38096821308135986
0.38039225339889526
0.43873852491378784
0.3907563090324402
0.3591312766075134
0.36643072962760925
0.3667054772377014
0.464578241109848
0.44710612297058105
0.3700430393218994
0.4197707176208496
0.4617459177970886
0.34730568528175354
0.402360200881958
0.4759341776371002
0.3781774938106537
0.4198276996612549
0.44099026918411255
0.3795541226863861
0.4393346309661865
0.3853849172592163
0.42261114716529846
0.3792082667350769
0.410521924495697
0.3458259105682373
0.3492019772529602
0.425506055355072
0.42241501808166504
0.4663676917552948
0.44577187299728394
0.3637714385986328
0.3725951910018921
0.3737543225288391
0.46589332818984985
0.3713909387588501
0.34792181849479675
0.47387123107910156
0.385734498500824
0.35619544982910156
0.36536723375320435
0.38496220111846924
0.3746044635772705
0.4077088236808777
0.42901134490966797
0.363161563873291
0.4003538489341736
0.39476826786994934
0.389088660478591

In [12]:
test_data_pred = predictor.predict(test_data, model='WeightedEnsemble_L2')
for item in test_data_pred:
    print(item)

Loading: ./agModels-body/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-body/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-body/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-body/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-body/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-body/models/WeightedEnsemble_L2/model.pkl


0.3720833957195282
0.36141401529312134
0.4118857979774475
0.3735848665237427
0.39341333508491516
0.4107148051261902
0.4215391278266907
0.4600366950035095
0.4157896637916565
0.36618757247924805
0.3986116051673889
0.4177819490432739
0.36982667446136475
0.429219514131546
0.41951000690460205
0.40689948201179504
0.3786681890487671
0.3794932961463928
0.3641781806945801
0.3630257844924927
0.4237116277217865
0.410045862197876
0.38450005650520325
0.4103325605392456
0.44294214248657227
0.38281816244125366
0.4273238182067871
0.39389359951019287
0.4268495440483093
0.39158082008361816
0.41073477268218994
0.37258750200271606
0.38087838888168335
0.42573630809783936
0.4052094519138336
0.3732624053955078
0.3703404664993286
0.3769264221191406
0.40301376581192017
0.39169740676879883
0.4110051095485687
0.3684316873550415
0.4364423453807831
0.385244220495224
0.4016824960708618
0.39341384172439575
0.40804240107536316
0.38155168294906616
0.34783822298049927
0.3813866376876831
0.4023202657699585
0.35021224617